# **PROJECT: Google Play Store Analytics**

# Loading the datasets

In [1]:
from google.colab import files
uploaded = files.upload()


Saving User Reviews.csv to User Reviews.csv
Saving Play Store Data.csv to Play Store Data.csv


# Task 1 – Grouped Bar Chart (Avg Rating vs Reviews)



TASK 1: Category Performance – Average Rating vs Total Reviews

**OBJECTIVE:**
Compare user satisfaction (average rating) with engagement
(total review count) for the top-performing app categories.

**DATASET:**
- Play Store Data.csv
- User Reviews.csv

**FILTERS APPLIED:**
- Average Rating ≥ 4.0
- App Size ≥ 10 MB
- Last Updated Month = January
- Top 10 Categories by Total Installs

**METRICS ANALYZED:**
- Average Rating (mean)
- Total Reviews (sum)

**TIME-BASED VISIBILITY:**
- Chart visible ONLY between 3:00 PM and 5:00 PM IST
- Outside this window, a “Task hidden” message is shown

**INSIGHT GOAL:**
Identify categories that combine high user satisfaction
with strong user engagement.



In [ ]:
# task1.py

from __future__ import annotations
import re
from pathlib import Path
from datetime import datetime, time
from zoneinfo import ZoneInfo

import numpy as np
import pandas as pd
import plotly.graph_objects as go

IST = ZoneInfo("Asia/Kolkata")
APP_CSV = Path("Play Store Data.csv")
REVIEWS_CSV = Path("User Reviews.csv")
OUT_HTML = Path("task1_grouped_bar.html")
ENFORCE_TIME_WINDOW = True              # flip to False to view anytime


def to_number_installs(x) -> float:
    """Parse installs like '1,000,000+' -> 1000000.0"""
    s = str(x)
    s = re.sub(r"[+,]", "", s)
    m = re.search(r"(\d+)", s)
    return float(m.group(1)) if m else np.nan

def size_to_mb(x) -> float:
    """Parse Size column to MB (supports K, M, G). 'Varies with device' -> NaN"""
    s = str(x).strip()
    if s.lower().startswith("varies") or s.lower() in {"nan", ""}:
        return np.nan
    m = re.match(r"([0-9]*\.?[0-9]+)\s*([kKmMgG]?)", s)
    if not m:
        return np.nan
    val = float(m.group(1)); unit = (m.group(2) or "").lower()
    if unit == "k": return val / 1024.0
    if unit == "g": return val * 1024.0
    return val

def within(start_h: int, start_m: int, end_h: int, end_m: int) -> bool:
    now = datetime.now(IST).time()
    return time(start_h, start_m) <= now <= time(end_h, end_m)


DO_HIDE = ENFORCE_TIME_WINDOW and not within(15, 0, 17, 0)  # 3–5 PM IST
if DO_HIDE:
    OUT_HTML.write_text("<h3>Task 1 hidden: visible only 3–5 PM IST.</h3>", encoding="utf-8")

# Load
apps = pd.read_csv(APP_CSV)
_ = pd.read_csv(REVIEWS_CSV)


df = apps.copy()

# robust existence checks
if "Installs" not in df.columns or "Reviews" not in df.columns or "Rating" not in df.columns:
    OUT_HTML.write_text("<h3>Required columns missing (Installs/Reviews/Rating).</h3>", encoding="utf-8")

df["InstallsNum"] = df.get("Installs", np.nan).apply(to_number_installs)
df["ReviewsNum"]  = pd.to_numeric(df.get("Reviews", np.nan), errors="coerce")
df["RatingNum"]   = pd.to_numeric(df.get("Rating", np.nan), errors="coerce")
df["SizeMB"]      = df.get("Size", np.nan).apply(size_to_mb)

# Dates
df["LastUpdated"] = pd.to_datetime(df.get("Last Updated", np.nan), errors="coerce")
df["LastUpdatedMonth"] = df["LastUpdated"].dt.month

# Filters
# Rating ≥ 4.0, Size ≥ 10 MB, Last update in January
t1 = df[
    (df["RatingNum"] >= 4.0) &
    (df["SizeMB"] >= 10) &
    (df["LastUpdatedMonth"] == 1)
].copy()


if DO_HIDE:
    print("Task 1 is hidden now; chart not rendered.")
else:

    if t1.empty or "Category" not in t1.columns:
        OUT_HTML.write_text("<h3>No data available after filters for Task 1.</h3>", encoding="utf-8")
    else:
        # Top 10 categories by total installs
        top10_idx = (
            t1.groupby("Category", dropna=True)["InstallsNum"]
              .sum()
              .sort_values(ascending=False)
              .head(10)
              .index
        )
        t1 = t1[t1["Category"].isin(top10_idx)].copy()

        # Aggregate for bars
        agg = (
            t1.groupby("Category", as_index=False)
              .agg(AvgRating=("RatingNum", "mean"),
                   TotalReviews=("ReviewsNum", "sum"))
        )

        if agg.empty:
            OUT_HTML.write_text("<h3>No data available after aggregation for Task 1.</h3>", encoding="utf-8")
        else:
            # Chart
            fig = go.Figure()

            fig.add_trace(go.Bar(
                x=agg["Category"], y=agg["AvgRating"],
                name="Avg Rating"
            ))

            fig.add_trace(go.Bar(
                x=agg["Category"], y=agg["TotalReviews"],
                name="Total Reviews", yaxis="y2"
            ))

            # Interactivity: sorting & visibility toggles
            order_by_rating  = agg.sort_values("AvgRating", ascending=False)["Category"]
            order_by_reviews = agg.sort_values("TotalReviews", ascending=False)["Category"]

            fig.update_layout(
                title="Avg Rating vs Total Reviews (Top 10 Categories by Installs — Jan, Rating≥4.0, Size≥10MB)",
                xaxis=dict(title="Category", categoryorder="array", categoryarray=list(order_by_reviews)),
                yaxis=dict(title="Avg Rating"),
                yaxis2=dict(title="Total Reviews", overlaying="y", side="right"),
                barmode="group",
                height=600,
                margin=dict(l=40, r=40, t=60, b=80),
                updatemenus=[dict(
                    type="dropdown",
                    x=1.0, xanchor="right", y=1.15, yanchor="top",
                    buttons=[
                        dict(label="Sort by Total Reviews",
                             method="relayout",
                             args=[{"xaxis.categoryorder":"array",
                                    "xaxis.categoryarray": list(order_by_reviews)}]),
                        dict(label="Sort by Avg Rating",
                             method="relayout",
                             args=[{"xaxis.categoryorder":"array",
                                    "xaxis.categoryarray": list(order_by_rating)}]),
                        dict(label="Show Both",
                             method="update",
                             args=[{"visible":[True, True]}]),
                        dict(label="Only Avg Rating",
                             method="update",
                             args=[{"visible":[True, False]}]),
                        dict(label="Only Total Reviews",
                             method="update",
                             args=[{"visible":[False, True]}]),
                    ]
                )]
            )

            # responsive HTML
            fig.write_html(
                str(OUT_HTML),
                include_plotlyjs="cdn",
                full_html=True,
                config={
                    "responsive": True,
                    "displaylogo": False,
                    "modeBarButtonsToRemove": [
                        "lasso2d", "select2d", "autoScale2d", "toggleSpikelines"
                    ]
                }
            )
            print(f"✅ Wrote {OUT_HTML}")


✅ Wrote task1_grouped_bar.html


In [ ]:
from google.colab import files
files.download("task1_grouped_bar.html")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Task 2 – CHOROPLETH MAP (GLOBAL INSTALLS)


TASK 2: Global Installs by Category – Choropleth Visualization

**OBJECTIVE:**
Visualize and compare global install volumes across the
top-performing app categories.

**IMPORTANT DATA CONSTRAINT:**
The dataset does NOT contain country or region information.
External datasets are NOT permitted.

**ANALYTICAL ASSUMPTION:**
- “Global installs” are interpreted as total worldwide installs
  aggregated across all apps within a category.
- The entire world is represented as a single geographic unit
  (ISO-3 code: WLD).
- Color intensity reflects magnitude, not geographic distribution.

# Caution #
Because the dataset contains no geographic identifiers, the choropleth uses a single real country polygon purely as a visual proxy.
Color encodes total global installs by category and does not imply geographic distribution. Regions without data remain uncolored by design.
There is no way to color all the countries without fabricating geographies which is outside the constraints of the task.


**FILTERS APPLIED:**
- Top 5 Categories by Total Installs
- Categories starting with A, C, G, or S are excluded
- Categories with installs ≥ 1 million are highlighted

**TIME-BASED VISIBILITY:**
- Chart visible ONLY between 6:00 PM and 8:00 PM IST

**INSIGHT GOAL:**
Provide a high-level comparison of global scale across
leading app categories under data constraints.



In [1]:

# TASK 2: Global Installs by Category (Choropleth – Visual Proxy)

# NOTE:
# - Dataset has no geographic information
# - External datasets are not allowed
# - Therefore, a SINGLE real country ("India") is used
#   purely as a VISUAL CONTAINER
# - Color encodes GLOBAL installs by category
# - Geography does NOT represent actual distribution


from pathlib import Path
from datetime import datetime, time
from zoneinfo import ZoneInfo
import pandas as pd
import re
import plotly.express as px


IST = ZoneInfo("Asia/Kolkata")
APP_CSV = Path("Play Store Data.csv")
OUT_HTML = Path("task2_choropleth.html")
ENFORCE_TIME_WINDOW = True  # set False to preview anytime


def within(h1, m1, h2, m2):
    now = datetime.now(IST).time()
    return time(h1, m1) <= now <= time(h2, m2)

if ENFORCE_TIME_WINDOW and not within(18, 0, 20, 0):
    OUT_HTML.write_text(
        "<h3>Task 2 hidden: visible only between 6–8 PM IST.</h3>",
        encoding="utf-8"
    )
    print("Task 2 hidden by time window.")
    exit()

# Load
df = pd.read_csv(APP_CSV)

df["Category"] = df["Category"].astype(str).str.strip()

def to_installs(x):
    if pd.isna(x):
        return 0.0
    return float(re.sub(r"[^\d]", "", str(x)) or 0)

df["InstallsNum"] = df["Installs"].apply(to_installs)

# Exclude categories starting with A, C, G, S
df = df[~df["Category"].str.match(r"^[ACGS]", case=False, na=False)]


agg = (
    df.groupby("Category", as_index=False)["InstallsNum"]
      .sum()
      .sort_values("InstallsNum", ascending=False)
)

top5 = agg.head(5).copy()

if top5.empty:
    OUT_HTML.write_text("<h3>No data after filters.</h3>", encoding="utf-8")
    exit()


# VISUAL PROXY ASSUMPTION

# A single real country ("India") is used ONLY as a drawable
# geometry for Plotly choropleth rendering.
# Color encodes GLOBAL installs by category.


top5["VisualCountry"] = "India"

default_cat = top5.iloc[0]["Category"]


fig = px.choropleth(
    top5[top5["Category"] == default_cat],
    locations="VisualCountry",
    locationmode="country names",
    color="InstallsNum",
    color_continuous_scale="Plasma",
    range_color=[0, top5["InstallsNum"].max()],
    title=f"Global Installs by Category (Visual Proxy): {default_cat}",
    labels={"InstallsNum": "Total Installs"}
)

# Dropdown

buttons = []
for _, row in top5.iterrows():
    label = row["Category"]
    if row["InstallsNum"] >= 1_000_000:
        label += " (≥1M installs)"

    buttons.append(dict(
        label=label,
        method="update",
        args=[
            {"z": [[row["InstallsNum"]]]},
            {"title": f"Global Installs by Category (Visual Proxy): {row['Category']}"}
        ]
    ))

fig.update_layout(
    updatemenus=[dict(
        type="dropdown",
        x=1.0, xanchor="right",
        y=1.15, yanchor="top",
        buttons=buttons
    )],
    margin=dict(l=40, r=40, t=90, b=40),
    height=600,
    coloraxis_colorbar=dict(title="Total Installs")
)


fig.write_html(
    str(OUT_HTML),
    include_plotlyjs="cdn",
    full_html=True,
    config={"responsive": True, "displaylogo": False}
)

print(f"✅ Task 2 choropleth written to {OUT_HTML}")


✅ Task 2 choropleth written to task2_choropleth.html


In [2]:
from google.colab import files
files.download("task2_choropleth.html")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Task 3 – DUAL-AXIS (FREE VS PAID)

TASK 3: Free vs Paid Apps – Installs vs Revenue Comparison

**OBJECTIVE:**
Analyze how free and paid apps differ in terms of
average installs and revenue generation across top categories.

**FILTERS APPLIED:**
- Top 3 Categories by Total Installs
- Installs ≥ 10,000
- Revenue ≥ $10,000
- Android Version > 4.0
- App Size ≥ 15 MB
- Content Rating = Everyone
- App name length ≤ 30 characters

**METRICS ANALYZED:**
- Average Installs
- Average Revenue (Price × Installs)


**TIME-BASED VISIBILITY:**
- Chart visible ONLY between 1:00 PM and 2:00 PM IST

**INSIGHT GOAL:**
Understand monetization and reach differences between
free and paid apps in high-performing categories.

In [ ]:
# task3_dual_axis.py
from __future__ import annotations
import re
from pathlib import Path
from datetime import datetime, time
from zoneinfo import ZoneInfo
import numpy as np
import pandas as pd
import plotly.graph_objects as go


IST = ZoneInfo("Asia/Kolkata")
APP_CSV = Path("Play Store Data.csv")
REVIEWS_CSV = Path("User Reviews.csv")
OUT_HTML = Path("task3_dual_axis.html")
ENFORCE_TIME_WINDOW = True  # flip to False to view anytime

def within(h1, m1, h2, m2):
    now = datetime.now(IST).time()
    return time(h1, m1) <= now <= time(h2, m2)

def to_installs(x):
    s = re.sub(r"[^\d]", "", str(x))
    return float(s) if s else np.nan

def to_size_mb(x):
    s = str(x).strip()
    if s.lower().startswith("varies") or s == "" or s.lower() == "nan": return np.nan
    m = re.match(r"([0-9.]+)\s*([kKmMgG]?)", s)
    if not m: return np.nan
    v = float(m.group(1)); u = (m.group(2) or "").lower()
    if u == "k": v /= 1024
    if u == "g": v *= 1024
    return v

def normalize_type(s):
    t = str(s).strip().lower()
    if t in {"free", "true", "0"}: return "Free"
    if t in {"paid", "false", "1"}: return "Paid"
    return "Paid" if t.startswith("paid") else ("Free" if t.startswith("free") else None)


DO_HIDE = ENFORCE_TIME_WINDOW and not within(13, 0, 14, 0)  # 1–2 PM IST
if DO_HIDE:
    OUT_HTML.write_text("<h3>Task 3 hidden: visible only 1–2 PM IST.</h3>", encoding="utf-8")

# Load
apps = pd.read_csv(APP_CSV)
_ = pd.read_csv(REVIEWS_CSV)


df = apps.copy()
df["InstallsNum"] = pd.to_numeric(df.get("Installs", "").apply(to_installs), errors="coerce")
df["SizeMB"]      = df.get("Size", "").apply(to_size_mb)
df["AndroidNum"]  = (df.get("Android Ver", "").astype(str)
                       .str.extract(r"([0-9]+(?:\.[0-9]+)?)", expand=False).astype(float))
df["AppNameLen"]  = df.get("App", "").astype(str).str.len()
df["CR_norm"]     = df.get("Content Rating", "").astype(str).str.strip().str.lower()

price = pd.to_numeric(df.get("Price", "").astype(str).str.replace("$","",regex=False), errors="coerce").fillna(0.0)
df["TypeNorm"] = df.get("Type", "").apply(normalize_type)
df["TypeNorm"] = np.where(df["TypeNorm"].isna(), np.where(price>0,"Paid","Free"), df["TypeNorm"])


if "Revenue" in df.columns:
    df["RevenueNum"] = pd.to_numeric(df["Revenue"], errors="coerce").fillna(0)
else:
    df["RevenueNum"] = np.where(df["TypeNorm"].eq("Paid"), price * df["InstallsNum"], 0.0)

if DO_HIDE:
    print("Task 3 is hidden now; chart not rendered.")
else:
    # Filters
    base = (
        (df["InstallsNum"] >= 10_000) &
        (pd.to_numeric(df["AndroidNum"], errors="coerce").fillna(0) > 4.0) &
        (pd.to_numeric(df["SizeMB"], errors="coerce").fillna(0) > 15) &
        (df["AppNameLen"] <= 30) &
        (df["CR_norm"].str.contains("^everyone", na=False))
    )
    paid_ok = df["TypeNorm"].eq("Paid") & (pd.to_numeric(df["RevenueNum"], errors="coerce").fillna(0) >= 10_000)
    free_ok = df["TypeNorm"].eq("Free")
    t3 = df[ base & (paid_ok | free_ok) ].copy().dropna(subset=["InstallsNum"])
    t3 = t3[t3["InstallsNum"] > 0]

    if t3.empty or "Category" not in t3.columns:
        OUT_HTML.write_text("<h3>No data after filters (or 'Category' missing) for Task 3.</h3>", encoding="utf-8")
    else:
        # Ensure we pick categories that have some Paid data
        check = (t3.groupby(["Category","TypeNorm"])["InstallsNum"].mean().unstack(fill_value=0))
        has_paid = check.get("Paid", 0) > 0
        candidates = check[has_paid].index.tolist()
        totals = (t3[t3["Category"].isin(candidates)]
                  .groupby("Category")["InstallsNum"].sum()
                  .sort_values(ascending=False))
        top3 = totals.head(3).index.tolist() if len(totals) else []

        if not top3:
            OUT_HTML.write_text("<h3>No categories with Paid data after filters.</h3>", encoding="utf-8")
        else:
            agg = (t3[t3["Category"].isin(top3)]
                     .groupby(["Category","TypeNorm"])
                     .agg(AvgInstalls=("InstallsNum","mean"),
                          AvgRevenue=("RevenueNum","mean"))
                     .reset_index())

            # Initial category
            cat0 = top3[0]
            sub0 = agg[agg["Category"] == cat0].copy()
            for t in ["Free","Paid"]:
                if t not in sub0["TypeNorm"].values:
                    sub0 = pd.concat([sub0, pd.DataFrame({"Category":[cat0],"TypeNorm":[t],"AvgInstalls":[0.0],"AvgRevenue":[0.0]})])
            sub0 = sub0.sort_values("TypeNorm")

            fig = go.Figure()
            fig.add_trace(go.Bar(x=sub0["TypeNorm"], y=sub0["AvgInstalls"], name="Avg Installs"))
            fig.add_trace(go.Scatter(x=sub0["TypeNorm"], y=sub0["AvgRevenue"], name="Avg Revenue",
                                     yaxis="y2", mode="lines+markers"))

            fig.update_layout(
                title=f"Free vs Paid — {cat0}: Avg Installs (bars) & Avg Revenue (line)",
                xaxis_title="Type",
                yaxis=dict(title="Avg Installs"),
                yaxis2=dict(title="Avg Revenue (USD)", overlaying="y", side="right"),
                height=560, margin=dict(l=40, r=40, t=80, b=60),
                legend=dict(orientation="h")
            )

            # Controls:
            # Dropdown #1: choose category
            cat_buttons = []
            for cat in top3:
                sub = agg[agg["Category"] == cat].copy()
                for t in ["Free","Paid"]:
                    if t not in sub["TypeNorm"].values:
                        sub = pd.concat([sub, pd.DataFrame({"Category":[cat],"TypeNorm":[t],"AvgInstalls":[0.0],"AvgRevenue":[0.0]})])
                sub = sub.sort_values("TypeNorm")
                cat_buttons.append(dict(
                    label=str(cat),
                    method="update",
                    args=[
                        {"x":[sub["TypeNorm"], sub["TypeNorm"]],
                         "y":[sub["AvgInstalls"], sub["AvgRevenue"]]},
                        {"title": f"Free vs Paid — {cat}: Avg Installs & Avg Revenue"}
                    ]
                ))

            # Dropdown #2: toggle which metrics to show
            metric_buttons = [
                dict(label="Show Both",    method="update", args=[{"visible":[True, True]}]),
                dict(label="Only Installs",method="update", args=[{"visible":[True, False]}]),
                dict(label="Only Revenue", method="update", args=[{"visible":[False, True]}]),
            ]

            fig.update_layout(
                updatemenus=[
                    # Dropdown #1
                    dict(
                        type="dropdown",
                        x=1.0, xanchor="right",
                        y=1.20, yanchor="top",
                        buttons=cat_buttons
                    ),
                    # Dropdown #2
                    dict(
                        type="dropdown",
                        x=1.0, xanchor="right",
                        y=1.07, yanchor="top",
                        buttons=metric_buttons
                    ),
                ]
            )


            fig.write_html(
                str(OUT_HTML),
                include_plotlyjs="cdn",
                full_html=True,
                config={"responsive": True, "displaylogo": False}
            )
            print(f"✅ Wrote {OUT_HTML}")


✅ Wrote task3_dual_axis.html


In [ ]:
from google.colab import files
files.download("task3_dual_axis.html")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Task 4 – TIME SERIES INSTALL TRENDS


TASK 4: Install Trends Over Time by App Category

**OBJECTIVE:**
Track how total installs evolve over time across selected
app categories and highlight periods of rapid growth.

**FILTERS APPLIED:**
- App name does NOT start with X, Y, or Z
- App name does NOT contain the letter 'S'
- Reviews > 500
- Categories starting with E, C, or B

**SPECIAL REQUIREMENTS:**
- Highlight periods where month-over-month install growth
  exceeds 20%
- Translate category names on the chart:
  • Beauty → Hindi
  • Business → Tamil
  • Dating → German


**TIME-BASED VISIBILITY:**
- Chart visible ONLY between 6:00 PM and 9:00 PM IST

**INSIGHT GOAL:**
Identify growth trends and seasonal spikes in installs
across specific app categories.




In [ ]:
# task4_time_series.py
from __future__ import annotations
import re
from pathlib import Path
from datetime import datetime, time
from zoneinfo import ZoneInfo
import numpy as np
import pandas as pd
import plotly.graph_objects as go


IST = ZoneInfo("Asia/Kolkata")
APP_CSV = Path("Play Store Data.csv")
REVIEWS_CSV = Path("User Reviews.csv")
OUT_HTML = Path("task4_time_series.html")
ENFORCE_TIME_WINDOW = True  # flip to False to preview anytime

def within(h1,m1,h2,m2):
    now = datetime.now(IST).time()
    return time(h1,m1) <= now <= time(h2,m2)

def to_installs(x):
    s = re.sub(r"[^\d]", "", str(x))
    return float(s) if s else np.nan

def translate_category(cat:str)->str:
    cl = cat.lower()
    if cl.startswith("beauty"):   return "सौंदर्य"
    if cl.startswith("business"): return "வணிகம்"
    if cl.startswith("dating"):   return "Partnersuche"
    return cat


DO_HIDE = ENFORCE_TIME_WINDOW and not within(18,0,21,0)
if DO_HIDE:
    OUT_HTML.write_text("<h3>Task 4 hidden: visible only 6–9 PM IST.</h3>", encoding="utf-8")

# Load
apps = pd.read_csv(APP_CSV)
_ = pd.read_csv(REVIEWS_CSV)

df = apps.copy()
df["InstallsNum"] = pd.to_numeric(df.get("Installs","").apply(to_installs), errors="coerce")
df["ReviewsNum"]  = pd.to_numeric(df.get("Reviews", np.nan), errors="coerce")
df["CategoryNorm"]= df.get("Category","").astype(str).str.strip()
df["AppName"]     = df.get("App","").astype(str)
df["LastUpdated"] = pd.to_datetime(df.get("Last Updated", np.nan), errors="coerce")
df["Month"]       = df["LastUpdated"].dt.to_period("M").dt.to_timestamp()

# Filters
cat_mask     = df["CategoryNorm"].str.match(r"^[ECB]", case=False, na=False)
no_xyz_start = ~df["AppName"].str.match(r"^[xyz]", case=False, na=False)
no_s_inside  = ~df["AppName"].str.contains("s", case=False, na=False)

f = df[cat_mask & no_xyz_start & no_s_inside &
       (df["ReviewsNum"]>500) &
       df["Month"].notna() & df["InstallsNum"].notna()].copy()

if DO_HIDE:
    print("Task 4 is hidden; no chart rendered.")
else:
    if f.empty:
        OUT_HTML.write_text("<h3>No data after filters for Task 4.</h3>", encoding="utf-8")
    else:
        # Monthly totals
        monthly = (f.groupby(["CategoryNorm","Month"])["InstallsNum"]
                     .sum().reset_index().sort_values(["CategoryNorm","Month"]))

        # Top-3 apps per category-month for hover
        contrib = (f.groupby(["CategoryNorm","Month","AppName"])["InstallsNum"]
                     .sum().reset_index())
        contrib["rank"] = contrib.groupby(["CategoryNorm","Month"])["InstallsNum"].rank("first", ascending=False)
        top3 = contrib[contrib["rank"] <= 3].copy()
        top3["AppLine"] = top3["AppName"] + " (" + top3["InstallsNum"].map(lambda v:f"{int(v):,}") + ")"
        hover_tbl = (top3.groupby(["CategoryNorm","Month"])["AppLine"]
                          .apply(lambda s:"<br>".join(s))
                          .reset_index().rename(columns={"AppLine":"TopApps"}))
        monthly = monthly.merge(hover_tbl, on=["CategoryNorm","Month"], how="left").fillna("No apps")

        # MoM > 20% shading
        mom = monthly.copy()
        mom["Prev"] = mom.groupby("CategoryNorm")["InstallsNum"].shift(1)
        mom["MoM"]  = (mom["InstallsNum"] - mom["Prev"])/mom["Prev"]


        fig = go.Figure()
        for cat, sub in monthly.groupby("CategoryNorm"):
            label = translate_category(cat)
            sub = sub.sort_values("Month")
            fig.add_trace(go.Scatter(
                x=sub["Month"], y=sub["InstallsNum"], mode="lines+markers",
                name=label,
                customdata=np.array(sub["TopApps"]).reshape(-1,1),
                hovertemplate=(
                    f"<b>{label}</b><br>"
                    "%{x|%b %Y}<br>"
                    "Installs: %{y:,.0f}"
                    "<br><br><b>Top Apps</b><br>%{customdata[0]}"
                    "<extra></extra>"
                )
            ))
            shade = mom[(mom["CategoryNorm"]==cat) & (mom["MoM"]>0.2) & mom["MoM"].notna()]["Month"]
            for m in shade:
                prev = pd.Timestamp(m)-pd.offsets.MonthBegin(1)
                fig.add_vrect(x0=prev, x1=pd.Timestamp(m),
                              fillcolor="lightgreen", opacity=0.25,
                              line_width=0, layer="below")

        # Range selector + slider
        fig.update_xaxes(
            rangeslider=dict(visible=True, thickness=0.12),
            rangeselector=dict(
                x=0, y=1.12,
                buttons=[
                    dict(count=3,label="3m",step="month",stepmode="backward"),
                    dict(count=6,label="6m",step="month",stepmode="backward"),
                    dict(step="all")
                ]
            ),
            automargin=True
        )
        fig.update_yaxes(automargin=True)


        cats = [translate_category(c) for c in monthly["CategoryNorm"].unique()]
        buttons = []
        for lab in cats + ["Show All"]:
            vis = [True]*len(fig.data) if lab=="Show All" else [(tr.name==lab) for tr in fig.data]
            buttons.append(dict(label=lab, method="update", args=[{"visible": vis}]))

        fig.update_layout(
            title=dict(
                text="Installs Over Time by Category (E/C/B) — Hover shows Top 3 Apps; shaded >20% MoM",
                y=0.975, x=0.5, xanchor="center", yanchor="top",
                pad=dict(t=20, b=0, l=0, r=0)
            ),
            xaxis_title="Month",
            yaxis_title="Total Installs",
            hovermode="x unified",
            legend=dict(
                title="App Category (Line Color)",
                orientation="h",
                yanchor="bottom", y=-0.32,
                xanchor="center",  x=0.5,
                bgcolor="rgba(255,255,255,0.8)",
                bordercolor="lightgray",
                borderwidth=1,
                itemsizing="trace",
                tracegroupgap=12
            ),
            height=760,
            margin=dict(l=50, r=50, t=150, b=140),
            updatemenus=[dict(
                type="dropdown",
                x=1.0, xanchor="right",
                y=1.22, yanchor="top",
                pad=dict(t=6, r=6, b=6, l=6),
                buttons=buttons,
                bgcolor="rgba(255,255,255,0.9)",
                bordercolor="lightgray",
                borderwidth=1
            )],
            paper_bgcolor="white",
            plot_bgcolor="white"
        )


        fig.write_html(
            str(OUT_HTML),
            include_plotlyjs="cdn",
            full_html=True,
            config={"responsive": True, "displaylogo": False}
        )
        print(f"✅ Wrote {OUT_HTML}")


✅ Wrote task4_time_series.html


In [ ]:
from google.colab import files
files.download("task4_time_series.html")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Task 5 – BUBBLE CHART (SIZE VS RATING)


TASK 5: App Size vs Rating – Bubble Chart Analysis

**OBJECTIVE:**
Examine the relationship between app size and user rating,
while accounting for popularity through install volume.

**FILTERS APPLIED:**
- Rating > 3.5
- Installs > 50,000
- Reviews > 500
- Sentiment Subjectivity > 0.5
- App name does NOT contain the letter 'S'
- Categories included:
  Game, Beauty, Business, Comics, Communication,
  Dating, Entertainment, Social, Events

**SPECIAL REQUIREMENTS:**
- Bubble size represents number of installs
- Game category highlighted in pink
- Translations applied on chart:
  • Beauty → Hindi
  • Business → Tamil
  • Dating → German

**TIME-BASED VISIBILITY:**
- Chart visible ONLY between 5:00 PM and 7:00 PM IST

**INSIGHT GOAL:**
Understand how app size correlates with user satisfaction
and popularity across categories.




In [ ]:
# task5_bubble.py
import re
from pathlib import Path
from datetime import datetime, time
from zoneinfo import ZoneInfo
import numpy as np
import pandas as pd
import plotly.express as px

IST = ZoneInfo("Asia/Kolkata")
APP_CSV = Path("Play Store Data.csv")
REVIEWS_CSV = Path("User Reviews.csv")
OUT_HTML = Path("task5_bubble.html")
ENFORCE_TIME_WINDOW = True # set to False to view anytime

def within(h1,m1,h2,m2):
    now = datetime.now(IST).time()
    return time(h1,m1) <= now <= time(h2,m2)

def to_num(x):
    s = re.sub(r"[^\d]", "", str(x))
    return float(s) if s else np.nan

def to_mb(x):
    s = str(x).strip()
    if s.lower().startswith("varies") or s=="" or s.lower()=="nan": return np.nan
    m = re.match(r"([0-9.]+)\s*([kKmMgG]?)", s)
    if not m: return np.nan
    v = float(m.group(1)); u = (m.group(2) or "").lower()
    if u=="k": v/=1024
    if u=="g": v*=1024
    return v

def translate(cat):
    cl = str(cat).lower()
    if cl.startswith("beauty"): return "सौंदर्य"
    if cl.startswith("business"): return "வணிகம்"
    if cl.startswith("dating"): return "Partnersuche"
    return str(cat)

DO_HIDE = ENFORCE_TIME_WINDOW and not within(17,0,19,0)
if DO_HIDE:
    OUT_HTML.write_text("<h3>Task 5 hidden: visible only 5–7 PM IST.</h3>", encoding="utf-8")

apps = pd.read_csv(APP_CSV)
rev  = pd.read_csv(REVIEWS_CSV)

# mean subjectivity per app
subj = (rev.groupby("App", dropna=True)["Sentiment_Subjectivity"].mean()
          .reset_index().rename(columns={"Sentiment_Subjectivity":"MeanSubjectivity"}))

df = apps.copy()
df["InstallsNum"] = pd.to_numeric(df.get("Installs","").apply(to_num), errors="coerce")
df["RatingNum"]   = pd.to_numeric(df.get("Rating", np.nan), errors="coerce")
df["SizeMB"]      = df.get("Size", np.nan).apply(to_mb)
df["ReviewsNum"]  = pd.to_numeric(df.get("Reviews", np.nan), errors="coerce")
df["CategoryNorm"]= df.get("Category","").astype(str).str.strip()
df["AppName"]     = df.get("App","").astype(str)
df = df.merge(subj, on="App", how="left")

# allowed categories
df["CatLower"] = df["CategoryNorm"].str.lower().replace({"commics":"comics","commication":"communication"})
wanted = {"game","beauty","business","comics","communication","dating","entertainment","social","events"}

mask = (
    (df["RatingNum"] > 3.5) &
    (df["CatLower"].isin(wanted)) &
    (df["ReviewsNum"] > 500) &
    (~df["AppName"].str.contains("s", case=False, na=False)) &
    (df["MeanSubjectivity"] > 0.5) &
    (df["InstallsNum"] >= 50_000)
)
data = df[mask].copy()

if DO_HIDE:
    print("Task 5 hidden; no chart rendered.")
else:
    if data.empty:
        OUT_HTML.write_text("<h3>No data after filters for Task 5.</h3>", encoding="utf-8")
    else:
        data["CategoryLabel"] = data["CategoryNorm"].apply(translate)
        fig = px.scatter(
            data, x="SizeMB", y="RatingNum",
            size="InstallsNum", color="CategoryLabel",
            hover_name="AppName",
            hover_data={"InstallsNum":":,","ReviewsNum":":,","MeanSubjectivity":":.2f","SizeMB":":.1f"},
            title="Size vs Rating (bubble = Installs) — subjectivity>0.5; Game highlighted"
        )
        # Game in pink
        for tr in fig.data:
            if "game" in tr.name.lower():
                tr.marker.update(color="#ff69b4")

        # category-isolate dropdown
        labels = sorted(data["CategoryLabel"].unique().tolist())
        label_to_mask = {lab: [(tr.name == lab) for tr in fig.data] for lab in labels}
        buttons = [dict(label="All Categories", method="update", args=[{"visible":[True]*len(fig.data)}])]
        for lab in labels:
            buttons.append(dict(label=lab, method="update", args=[{"visible": label_to_mask[lab]}]))
        fig.update_layout(updatemenus=[dict(type="dropdown", x=1.0, xanchor="right", y=1.15, yanchor="top", buttons=buttons)])

        fig.update_layout(xaxis_title="App Size (MB)", yaxis_title="Average Rating", legend_orientation="h", height=650)

        fig.write_html(str(OUT_HTML), include_plotlyjs="cdn", full_html=True,
                       config={"responsive": True, "displaylogo": False})
        print(f"✅ Wrote {OUT_HTML}")


✅ Wrote task5_bubble.html


In [ ]:
from google.colab import files
files.download("task5_bubble.html")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Task 6 – STACKED AREA (CUMULATIVE INSTALLS)


TASK 6: Cumulative Installs Over Time – Stacked Area Chart

**OBJECTIVE:**
Visualize cumulative install growth over time for
selected app categories.

**FILTERS APPLIED:**
- Average Rating ≥ 4.2
- App name does NOT contain numbers
- Categories starting with T or P
- Reviews > 1,000
- App Size between 20 MB and 80 MB

**SPECIAL REQUIREMENTS:**
- Cumulative stacked area chart
- Highlight months where install growth exceeds 25% MoM
- Legend translations:
  • Travel & Local → French
  • Productivity → Spanish
  • Photography → Japanese

**TIME-BASED VISIBILITY:**
- Chart visible ONLY between 4:00 PM and 6:00 PM IST

**INSIGHT GOAL:**
Identify sustained growth patterns and high-impact periods
across top-performing app categories.



In [ ]:
# task6_stacked_area.py
import re
from pathlib import Path
from datetime import datetime, time
from zoneinfo import ZoneInfo
import numpy as np
import pandas as pd
import plotly.graph_objects as go

IST = ZoneInfo("Asia/Kolkata")
APP_CSV = Path("Play Store Data.csv")
REVIEWS_CSV = Path("User Reviews.csv")
OUT_HTML = Path("task6_stacked_area.html")
ENFORCE_TIME_WINDOW = True  # set to False to view anytime

def within(h1, m1, h2, m2):
    now = datetime.now(IST).time()
    return time(h1, m1) <= now <= time(h2, m2)

def to_num(x):
    s = re.sub(r"[^\d]", "", str(x))
    return float(s) if s else np.nan

def to_mb(x):
    s = str(x).strip()
    if s.lower().startswith("varies") or s == "" or s.lower() == "nan":
        return np.nan
    m = re.match(r"([0-9.]+)\s*([kKmMgG]?)", s)
    if not m:
        return np.nan
    v = float(m.group(1)); u = (m.group(2) or "").lower()
    if u == "k": v /= 1024
    if u == "g": v *= 1024
    return v

def translate(cat):
    cl = str(cat).lower()
    if cl.startswith("travel") or "travel & local" in cl: return "Voyage et local"
    if cl.startswith("productivity"): return "Productividad"
    if cl.startswith("photography"): return "写真"
    return str(cat)


DO_HIDE = ENFORCE_TIME_WINDOW and not within(16, 0, 18, 0)
if DO_HIDE:
    OUT_HTML.write_text("<h3>Task 6 hidden: visible only 4–6 PM IST.</h3>", encoding="utf-8")

# Load
apps = pd.read_csv(APP_CSV)
_ = pd.read_csv(REVIEWS_CSV)


df = apps.copy()
df["InstallsNum"] = pd.to_numeric(df.get("Installs","").apply(to_num), errors="coerce")
df["RatingNum"]   = pd.to_numeric(df.get("Rating", np.nan), errors="coerce")
df["SizeMB"]      = df.get("Size", np.nan).apply(to_mb)
df["ReviewsNum"]  = pd.to_numeric(df.get("Reviews", np.nan), errors="coerce")
df["AppName"]     = df.get("App","").astype(str)
df["CategoryNorm"]= df.get("Category","").astype(str).str.strip()
df["LastUpdated"] = pd.to_datetime(df.get("Last Updated", np.nan), errors="coerce")
df["Month"]       = df["LastUpdated"].dt.to_period("M").dt.to_timestamp()

mask = (
    (df["RatingNum"] >= 4.2) &
    (~df["AppName"].str.contains(r"\d", regex=True, na=False)) &
    (df["CategoryNorm"].str.match(r"^[TP]", case=False, na=False)) &
    (df["ReviewsNum"] > 1000) &
    (df["SizeMB"].between(20, 80)) &
    df["Month"].notna() & df["InstallsNum"].notna()
)
f = df[mask].copy()

if DO_HIDE:
    print("Task 6 hidden; no chart rendered.")
else:
    if f.empty:
        OUT_HTML.write_text("<h3>No data after filters for Task 6.</h3>", encoding="utf-8")
    else:
        # Monthly sums
        monthly = (f.groupby(["CategoryNorm","Month"])["InstallsNum"]
                     .sum().reset_index().sort_values(["CategoryNorm","Month"]))

        mon_pivot = (monthly.pivot(index="Month", columns="CategoryNorm", values="InstallsNum")
                            .sort_index().fillna(0))
        cum_pivot = mon_pivot.cumsum()

        # Initial (cumulative stacked)
        fig = go.Figure()
        for col in cum_pivot.columns:
            fig.add_trace(go.Scatter(
                x=cum_pivot.index, y=cum_pivot[col].values,
                mode="lines", stackgroup="one", name=translate(col)
            ))


        def traces_for(df_in, stacked: bool):
            traces = []
            for col in df_in.columns:
                traces.append(dict(
                    x=df_in.index, y=df_in[col].values,
                    mode="lines",
                    stackgroup="one" if stacked else None,
                    name=translate(col)
                ))
            return traces

        # Shade months with >25% MoM increase
        mo = monthly.copy()
        mo["Prev"] = mo.groupby("CategoryNorm")["InstallsNum"].shift(1)
        mo["MoM"]  = (mo["InstallsNum"] - mo["Prev"]) / mo["Prev"]
        months_to_shade = sorted(mo.loc[(mo["MoM"] > 0.25) & mo["MoM"].notna(), "Month"].unique())
        for m in months_to_shade:
            prev = pd.Timestamp(m) - pd.offsets.MonthBegin(1)
            fig.add_vrect(x0=prev, x1=pd.Timestamp(m), fillcolor="orange",
                          opacity=0.18, line_width=0, layer="below")

        # --- Toggle + Dropdown menus
        toggle_menu = dict(
            type="buttons",
            x=1.0, xanchor="right",
            y=1.22, yanchor="top",
            pad=dict(t=4, r=6, l=6, b=4),
            bgcolor="rgba(255,255,255,0.9)",
            bordercolor="lightgray",
            borderwidth=1,
            buttons=[
                dict(label="Cumulative (Stacked)", method="update",
                     args=[{"data": traces_for(cum_pivot, True)}]),
                dict(label="Monthly (Lines)", method="update",
                     args=[{"data": traces_for(mon_pivot, False)}]),
            ]
        )

        labels = [translate(c) for c in mon_pivot.columns]
        def build_vis(label):
            return [True if label == "Show All" else (tr.name == label) for tr in fig.data]

        dropdown_buttons = [dict(label="Show All", method="update",
                                 args=[{"visible": build_vis("Show All")}])]
        for lab in labels:
            dropdown_buttons.append(dict(label=lab, method="update",
                                         args=[{"visible": build_vis(lab)}]))

        dropdown_menu = dict(
            type="dropdown",
            x=1.0, xanchor="right",
            y=1.09, yanchor="top",
            pad=dict(t=4, r=6, l=6, b=4),
            bgcolor="rgba(255,255,255,0.9)",
            bordercolor="lightgray",
            borderwidth=1,
            buttons=dropdown_buttons
        )

        fig.update_layout(
            updatemenus=[toggle_menu, dropdown_menu],
            title=dict(
                text="Cumulative Installs Over Time — Stacked by Category (toggle Monthly/Cumulative)",
                y=0.975, x=0.5, xanchor="center", yanchor="top",
                pad=dict(t=20)
            ),
            xaxis_title="Month",
            yaxis_title="Installs",
            legend=dict(
                orientation="h",
                yanchor="bottom", y=-0.25,
                xanchor="center", x=0.5,
                bgcolor="rgba(255,255,255,0.8)",
                bordercolor="lightgray",
                borderwidth=1
            ),
            height=700,
            margin=dict(l=50, r=50, t=150, b=120),
            paper_bgcolor="white",
            plot_bgcolor="white"
        )


        fig.write_html(
            str(OUT_HTML),
            include_plotlyjs="cdn",
            full_html=True,
            config={"responsive": True, "displaylogo": False}
        )
        print(f"✅ Wrote {OUT_HTML}")


✅ Wrote task6_stacked_area.html


In [ ]:
from google.colab import files
files.download("task6_stacked_area.html")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>